In [85]:
import pandas as pd
import datetime as dt

def conv_str_to_time(time_str):
    time = dt.datetime.strptime(time_str, '%Y.%m.%d %H:%M:%S.%f')
    return time

#Reading the data
df = pd.read_csv('17_04_NT_deals.csv', delimiter=';', index_col='Timestamp')
column_to_del = []

#Filter the data
for column in df.columns:
    if (column != 'Counterparty' and column != 'B/S' and column !='Instrument' and \
        column !='Tenor' and column != 'Price' and column != 'Amount' and \
        column !='Counterparty Trader' and column != 'Initiator Id' and \
        column != 'Initiator' and column != 'Timestamp'):
            column_to_del.append(column)

df = df.drop(columns=column_to_del, axis=1)    

df = df[df['Instrument'] == 'USDRUB']
df = df[df['Tenor'] == 'TOM']
df = df[df['Counterparty'] != 'ALFN']

df.reset_index(inplace=True)

#change B/S to 1/-1
dict_b_s = {'Buy': 1, 'Sell': -1}
df['B/S'] = df['B/S'].map(dict_b_s)

#Creating Volume*Side B/S
df['Amount_b/s'] = df['B/S']*df['Amount']

#Deleting LPs (need only external trades)
i=0
index_to_del = []
for i in range(df.shape[0]):
    counterparty = df['Counterparty Trader'].iloc[i]
    if counterparty.endswith('LP'):
        index_to_del.append(i)

df = df.drop(labels=index_to_del, axis=0)
df.sort_values(by='Timestamp', axis=0, ascending=True, inplace=True)

# Particular client's under investigation trades
client_trades = df[df['Counterparty'] == 'IMBX']

#Other external trades
ext_trades = df[df['Counterparty'] != 'IMBX']

In [87]:

time_of_ext_trades = []
volumes_ext =[]
time_and_vol_ext = []

time_of_client_trades = []
volumes_client = []
time_and_vol_client = []

#going through Clients's trades to analyze
#for each trade time we build range t0, t0+600 sec 
#within this range we build volumes

for j in range(client_trades.shape[0]):
    time_0 = conv_str_to_time(client_trades['Timestamp'].iloc[j])
    delta = dt.timedelta(seconds=600)
    time_1 = time_0 + delta

    time_str_0 = time_0.strftime('%Y.%m.%d %H:%M:%S')
    time_str_1 = time_1.strftime('%Y.%m.%d %H:%M:%S')
    
    #selecting trades in time range [t0, t0+600 sec]
    client_select = client_trades[client_trades['Timestamp'] >= time_str_0]
    client_select = client_select[client_select['Timestamp'] <= time_str_1]
    ext_select = ext_trades[ext_trades['Timestamp'] >= time_str_0]
    ext_select = ext_select[ext_select['Timestamp'] <= time_str_1]

    for i in range(ext_select.shape[0]):
        time_delta = (conv_str_to_time(ext_select['Timestamp'].iloc[i]) - time_0).total_seconds()
        time_of_ext_trades.append(time_delta)
        volumes_ext.append(ext_select['Amount_b/s'].iloc[i])
    time_and_vol_ext.append((time_of_ext_trades, volumes_ext))

    for i in range(client_select.shape[0]):
        time_delta = (conv_str_to_time(client_select['Timestamp'].iloc[i]) - time_0).total_seconds()
        time_of_client_trades.append(time_delta)
        volumes_client.append(client_select['Amount_b/s'].iloc[i])
    time_and_vol_client = ((time_of_client_trades, volumes_client))

    #exclude selected trades (not to duplicate in next step of calculation)
    ext_trades = ext_trades[~ext_trades['Timestamp'].isin(ext_select)]
    client_trades = client_trades[~client_trades['Timestamp'].isin(client_select)]

In [102]:
volume = 0
volume_total = []
for vol in time_and_vol_ext[0][1]:
    volume = volume +vol
    volume_total.append(round(volume, 2))
    
print(volume_total)

[-1000000.0, -2000000.0, -2500000.0, -2000000.0, -1500000.0, -500000.0, -1352892.0, -1852892.0, -2352892.0, -3352892.0, -3402892.0, -2402892.0, -1402892.0, -902892.0, -908892.0, 91108.0, 5091108.0, 4639108.0, 4589108.0, 4449108.0, 4049108.0, 3549245.0, 3049245.0, 3037245.0, 3016245.0, 2516245.0, 2449245.0, -556755.0, -506755.0, -1506755.0, -1206755.0, -1006755.0, -2006755.0, -1606755.0, -1106755.0, -1606755.0, -1656755.0, -1156755.0, 8843245.0, 8718245.0, 8218245.0, 7218300.0, 6718300.0, 5718300.0, 5218300.0, 5368300.0, 6368300.0, 6168300.0, 5668300.0, 5168300.0, 4868300.0, 4768300.0, 5768300.0, 2768300.0, 3268300.0, 2268300.0, 3268300.0, 3768300.0, 4268300.0, 4768300.0, 5018300.0, 6018300.0, 5518300.0, 4518300.0, 3518300.0, 4518300.0, -481700.0, -1481700.0, -2481700.0, -3481700.0, -3384700.0, -2884700.0, -2715830.0, -2215830.0, -1715830.0, -1215830.0, -215830.0, -715830.0, -865762.0, -1365762.0, -1865762.0, -2365762.0, -2865762.0, -3365762.0, -3865762.0, -4365762.0, -3865762.0, -28657

In [47]:
time_0 = conv_str_to_time(imbx_trades['Timestamp'].iloc[0])
time_1 = conv_str_to_time(imbx_trades['Timestamp'].iloc[1])
delta = time_1 - time_0
print(delta)


1:21:39.584000


In [49]:
delta = (conv_str_to_time(ext_trades['Timestamp'][1]) - conv_str_to_time(ext_trades['Timestamp'][0]))
print(delta)

0:15:20.473000


In [13]:
dt.timedelta(seconds=600)

datetime.timedelta(0, 600)

In [35]:
print(imbx_trades)

                    Timestamp Counterparty   B/S Instrument Tenor    Price  \
150   2019.04.17 10:10:45.862         IMBX  Sell     USDRUB   TOM  63.9582   
385   2019.04.17 11:32:25.446         IMBX  Sell     USDRUB   TOM  63.8774   
394   2019.04.17 11:38:21.738         IMBX  Sell     USDRUB   TOM  63.9145   
1991  2019.04.17 13:18:07.916         IMBX   Buy     USDRUB   TOM  63.9314   
1421  2019.04.17 17:45:05.197         IMBX  Sell     USDRUB   TOM  63.9057   

          Amount Counterparty Trader  Initiator Id Initiator  
150    5000000.0               IMBX3          4181     IMBX3  
385    5000000.0               IMBX3          4181     IMBX3  
394   10000000.0               IBMX2          1729     IBMX2  
1991  10000000.0               IMBX3          4181     IMBX3  
1421  20000000.0               IBMX2          1729     IBMX2  


In [44]:
ext_trades[~ext_trades['Timestamp'].isin(ext_select)]

count     -2556
unique       -2
top          -1
freq      -2556
Name: Timestamp, dtype: object

In [41]:
ext_select['Timestamp']

151     2019.04.17 10:10:46.348
152     2019.04.17 10:11:11.894
153     2019.04.17 10:11:36.555
1526    2019.04.17 10:11:37.850
1527    2019.04.17 10:11:37.850
1528    2019.04.17 10:12:01.466
154     2019.04.17 10:12:05.421
155     2019.04.17 10:12:05.425
156     2019.04.17 10:12:05.472
157     2019.04.17 10:12:05.660
158     2019.04.17 10:12:14.483
1529    2019.04.17 10:12:15.448
1530    2019.04.17 10:12:23.435
1531    2019.04.17 10:12:28.734
159     2019.04.17 10:12:30.968
1532    2019.04.17 10:12:42.558
1533    2019.04.17 10:12:45.718
161     2019.04.17 10:12:45.926
162     2019.04.17 10:12:46.600
163     2019.04.17 10:12:49.421
164     2019.04.17 10:12:53.099
166     2019.04.17 10:12:56.199
165     2019.04.17 10:12:56.199
167     2019.04.17 10:13:06.108
168     2019.04.17 10:13:06.221
169     2019.04.17 10:13:06.685
170     2019.04.17 10:13:12.158
171     2019.04.17 10:13:14.948
1534    2019.04.17 10:13:16.309
172     2019.04.17 10:13:20.650
                 ...           
1566    

In [54]:
print(df)

                    Timestamp Counterparty  B/S Instrument Tenor      Price  \
0     2019.04.17 02:37:20.828         ASAN   -1     USDRUB   TOM  64.100000   
1     2019.04.17 02:52:41.301         RZBM   -1     USDRUB   TOM  64.100500   
1445  2019.04.17 04:10:31.691         OFCB    1     USDRUB   TOM  64.035900   
2     2019.04.17 04:38:08.137         ASAN   -1     USDRUB   TOM  64.100500   
3     2019.04.17 05:06:36.963         ASAN   -1     USDRUB   TOM  64.100500   
4     2019.04.17 05:11:47.545         PRMC   -1     USDRUB   TOM  64.050500   
1446  2019.04.17 05:46:14.520         ALFA    1     USDRUB   TOM  64.035900   
1447  2019.04.17 05:46:18.432         ALFA    1     USDRUB   TOM  64.035900   
5     2019.04.17 05:50:48.789         ASAN   -1     USDRUB   TOM  64.100000   
1448  2019.04.17 06:23:26.735         OFCB    1     USDRUB   TOM  64.064100   
6     2019.04.17 06:35:47.957         TCBM   -1     USDRUB   TOM  64.102000   
1449  2019.04.17 06:51:16.357         ALFA    1     